# Goal: Create a prototype web tool that turns a list of Simbad-resolvable astronomical object names into a properly formatted Magellan catalog entry.

## Imports

In [ ]:
import numpy as np
import pandas as pd 
from astroquery.simbad import Simbad
from subprocess import *

## Objects & Guide Stars & Instrument Rotator & Equinox & Epoch of PM

In [ ]:
target_list = ['Vega', 'Sirius', 'HD 168607', 'HD 167356', 'HD 168625']

In [ ]:
guide_stars = ['HD 200466B', 'HD 207966']

In [ ]:
InstRotOffset = -140.75
InstRotOffsetMode = 'OFF'

In [ ]:
equinox = 2000

In [ ]:
epoch_pm = 2019.7

## Magellan Catalog Entry

In [ ]:
MagCatEntry = pd.DataFrame(columns = ['#',  
                                      ' ', 
                                      'RA', 
                                      'Dec', 
                                      'equinox', 
                                      'RApm', 
                                      'Decpm', 
                                      'offset', 
                                      'rot', 
                                      'RA_probe1', 
                                      'Dec_probe1', 
                                      'equinox_probe1', 
                                      'RA_probe2', 
                                      'Dec_probe2', 
                                      'equinox_probe2', 
                                      'pm_epoch'], 
                           index = [' '])

#Appending Header
MagCatEntry.at['0', '#'] = '###'
MagCatEntry.at['0', ' '] = 'name'
MagCatEntry.at['0', 'RA'] = 'hh:mm:ss.s'
MagCatEntry.at['0', 'Dec'] = 'sdd:mm:ss'
MagCatEntry.at['0', 'equinox'] = 'yyyy.0'
MagCatEntry.at['0', 'RApm'] = 's.ss'
MagCatEntry.at['0', 'Decpm'] = 's.ss'
MagCatEntry.at['0', 'equinox'] = 'yyyy.0'
MagCatEntry.at['0', 'offset'] = 'angle'
MagCatEntry.at['0', 'rot'] = 'mode'
MagCatEntry.at['0', 'RA_probe1'] = 'hh:mm:ss.s'
MagCatEntry.at['0', 'Dec_probe1'] = 'sdd:mm:ss'
MagCatEntry.at['0', 'equinox_probe1'] = 'yyyy.0'
MagCatEntry.at['0', 'RA_probe2'] = 'hh:mm:ss.s'
MagCatEntry.at['0', 'Dec_probe2'] = 'sdd:mm:ss'
MagCatEntry.at['0', 'equinox_probe2'] = 'yyyy.0'
MagCatEntry.at['0', 'pm_epoch'] = 'yyyy.0'


x = 0
for i in target_list:
    MagCatEntry = MagCatEntry.append(pd.Series(), ignore_index = True)
    x = x + 1

MagCatEntry = MagCatEntry.drop([0])
MagCatEntry = MagCatEntry.reset_index()
MagCatEntry = MagCatEntry.drop(columns=['index'])

In [ ]:
MagCatEntry

## Custom Simbad

In [ ]:
customSimbad = Simbad()
customSimbad.add_votable_fields('ra(s)')
customSimbad.add_votable_fields('dec(s)')
customSimbad.add_votable_fields('pmra')
customSimbad.add_votable_fields('pmdec')

In [ ]:
#Simbad.list_votable_fields()

In [ ]:
#Simbad.get_field_description('propermotions')

## Fields 1 – 7

In [ ]:
columns = {'MAIN_ID': [0], 'RA': [0], 'DEC': [0], 'PMRA': [0], 'PMDEC': [0]}
targets = pd.DataFrame(data = columns)

x = 0
for i in target_list:
    star = customSimbad.query_object(target_list[x])
    star.keep_columns(['MAIN_ID', 'RA', 'DEC', 'PMRA', 'PMDEC'])
    star = star.to_pandas()
    targets = targets.append(star)
    x = x + 1
    
    
targets = targets.iloc[1:]

In [ ]:
targets

In [ ]:
targets = targets.values.tolist()

In [ ]:
targets

## Fields 10 – 15

In [ ]:
columns = {'RA': [0], 'DEC': [0]}
guides = pd.DataFrame(data = columns)

x = 0
for i in guide_stars:
    star = customSimbad.query_object(guide_stars[x])
    star.keep_columns(['RA', 'DEC'])
    star = star.to_pandas()
    guides = guides.append(star)
    x = x + 1
    
    
guides = guides.iloc[1:]

In [ ]:
guides

In [ ]:
guides = guides.values.tolist()

## Appending the Series

### Appending Rows 5, 8 – 16 (static)

In [ ]:
x = 1
for i in target_list:
    MagCatEntry.at[x, 'equinox'] = equinox
    MagCatEntry.at[x, 'offset'] = InstRotOffset
    MagCatEntry.at[x, 'rot'] = InstRotOffsetMode
    MagCatEntry.at[x, 'RA_probe1'] = guides[0][0]
    MagCatEntry.at[x, 'Dec_probe1'] = guides[0][1]
    MagCatEntry.at[x, 'equinox_probe1'] = equinox
    MagCatEntry.at[x, 'RA_probe2'] = guides[1][0]
    MagCatEntry.at[x, 'Dec_probe2'] = guides[1][1]
    MagCatEntry.at[x, 'equinox_probe2'] = equinox
    MagCatEntry.at[x, 'pm_epoch'] = epoch_pm
    x = x + 1

In [ ]:
MagCatEntry

### Appending Rows 1-4

In [ ]:
x = 1
y = 0
for i in target_list:
    MagCatEntry.at[x, '#'] = '{0:03}'.format(x)
    MagCatEntry.at[x, ' '] = targets[y][0].decode('utf-8')
    MagCatEntry.at[x, 'RA'] = targets[y][1]
    MagCatEntry.at[x, 'Dec'] = targets[y][2]
    MagCatEntry.at[x, 'equinox'] = equinox
    MagCatEntry.at[x, 'RApm'] = targets[y][3]
    MagCatEntry.at[x, 'Decpm'] = targets[y][4]
    x = x + 1
    y = y + 1

In [ ]:
MagCatEntry

## Saving as .txt

In [ ]:
np.savetxt('catalog.txt', MagCatEntry, fmt="%-15s", header = '                              RA              Dec             equinox         RApm            Decpm           offset          rot             RA_probe1       Dec_probe1      equinox_probe1  RA_probe2       Dec_probe2      equinox_probe2  pm_epoch')